Загрузить данные из таблицы test.xlsx.  
Проанализировать текст в колонке text, используя метод обработки текста.  
Сгенерировать вероятный краткий ответ из всего текста (состоящий из нескольких слов) и записать его в колонку answer.  

Пример:  
Текст1 - Удовлетворить  
Текст2 - Производство окончено  
Текст3 - Отказ, ИП приостановлено  
Текст4 - Отказ, невозможно установить местонахождение должника  

In [1]:
import pandas as pd
import os

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

os.chdir('../../')

In [2]:
df = pd.read_csv('data/processed/rule_classified.csv')

In [3]:
df_labeled = df[df['answer'].notna()].copy()
# original_indices_labeled = df_labeled.index.copy() # Для последующего объеденения с предсказаниями
df_labeled = df_labeled.reset_index(drop=True)

df_labeled['answer'].value_counts()

answer
Отказ                                     217
Частично удовлетворено                    138
Обращение рассмотрено                     130
Взыскание обращено                        106
Запрос направлен                           80
Возбуждено исполнительное производство     76
Постановление вынесено                     37
Удовлетворено                              27
Заявления и жалобы рассматриваются         26
Объявлен исполнительный розыск             20
Применены меры для исполнения              19
Запрет действий                            14
Name: count, dtype: int64

In [4]:
from src.classification_utils import split_data, encode_labels

X_train, X_test, y_train, y_test = split_data(df_labeled)
y_train_enc, y_test_enc, le = encode_labels(y_train, y_test)

print(f"{'='*15}Размеры классов тренировочной выборки:{'='*15}\n{y_train.value_counts()}\n")
print(f"{'='*15}Размеры классов тестовой выборки:{'='*15}\n{y_test.value_counts()}")

===============Размеры классов тренировочной выборки:===============
answer
Отказ                                     152
Частично удовлетворено                     97
Обращение рассмотрено                      91
Взыскание обращено                         74
Запрос направлен                           56
Возбуждено исполнительное производство     53
Постановление вынесено                     26
Удовлетворено                              19
Заявления и жалобы рассматриваются         18
Объявлен исполнительный розыск             14
Применены меры для исполнения              13
Запрет действий                            10
Name: count, dtype: int64

===============Размеры классов тестовой выборки:===============
answer
Отказ                                     65
Частично удовлетворено                    41
Обращение рассмотрено                     39
Взыскание обращено                        32
Запрос направлен                          24
Возбуждено исполнительное производство    23
Пост

In [5]:
import pickle
from src.classifiers import get_classifiers
from src.resamplers import get_resamplers

# Получаем кортеж векторизированных данных
with open('vectors/vectorizers.pkl', 'rb') as f:
    vectorizers = pickle.load(f)

# Получаем список моделей и параметры их обучения
classifiers = get_classifiers(le)
resamplers = get_resamplers()

Так как тексты юридические, мы не можем аугментировать данные через синонимы, перефразировки, двойные переводы.  
Остаются только методы балансировки по типу SMOTE и создание новых примеров через LLM.  
Проблема в SMOTE, что он ищет ближайших соседей примеров. То есть с самыми редкими классами <10 примеров SMOTE может создать очень похожие или некачественные данные.  
Есть специальный SMOTE для текстов - [SMOTExT](https://arxiv.org/pdf/2505.13434), но его нету в открытом доступе, только статья 2025 года. Это SMOTE, но с генерацией новых примеров через эмбеддинги BERT.

Если бы я заранее не искал классы через кластеризацию, то можно было бы применить oversampling на основе сходства, но оно всё равно бы не увеличило количество классов, а попыталось бы прикрепить к классам близкие примеры.  

In [6]:
from src.classification_utils import load_np_arrays, crossval_report, log_report
from imblearn.pipeline import Pipeline as ImbPipeline
from tqdm.notebook import tqdm
import mlflow
import warnings
import os

os.environ['MLFLOW_SUPPRESS_PRINTING_URL_TO_STDOUT'] = '1' # так как выводит view run и view experiment из-за того, что запустил отдельный сервер
warnings.filterwarnings('ignore')

mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment('text_classification')

pbar_vectorizers = tqdm(total=len(vectorizers), desc="Векторизаторы", position=0)
pbar_resamplers = tqdm(total=len(resamplers), desc="Балансировщики", position=1)
pbar_classifiers = tqdm(total=len(classifiers), desc="Классификаторы", position=2)

for vec_name, (train_path, test_path) in vectorizers.items():
    X_train_vec, X_test_vec = load_np_arrays(train_path, test_path)

    pbar_vectorizers.set_description(f"Векторизатор {vec_name}")
    pbar_resamplers.reset()

    for resampler_name, resampler in resamplers:
        pbar_resamplers.set_description(f"Балансировщик {resampler_name}")
        pbar_classifiers.reset()
        
        for classifier_name, classifier in classifiers:
            pipeline = ImbPipeline([
                ('resampler', resampler),
                ('classifier', classifier)
            ])

            report = crossval_report(classifier, X_train_vec, y_train, y_train_enc, le, pipeline=pipeline)
            log_report(report, vec_name, classifier_name, resampler_name)

            pbar_classifiers.set_description(f"Классификатор {classifier_name}")
            pbar_classifiers.update(1)

        pbar_resamplers.update(1)

    pbar_vectorizers.update(1)

Векторизаторы:   0%|          | 0/6 [00:00<?, ?it/s]

Балансировщики:   0%|          | 0/5 [00:00<?, ?it/s]

Классификаторы:   0%|          | 0/8 [00:00<?, ?it/s]

72 минут в общем